
> **Rodar este notebook no ambiente pago da databricks**

### Manipulação de Dados com DataFrames no PySpark - write

A função **`write`** em PySpark é usada para salvar um DataFrame em um arquivo ou tabela. A estrutura geral é:

```
(
dataframe
    .write
    .option("option_name", "option_value")
    .format("file_format")
    .mode("mode")
    .save("path")
)
```

#### Estrutura Geral

1. **`df.write`**: Inicia a operação de escrita do DataFrame.
2. **`option("option_name", "option_value")`**: Define opções específicas para o formato de saída (ex.: `header` para CSV, `compression` para Parquet).
3. **`format("file_format")`**: Define o formato do arquivo de saída (ex.: `"csv"`, `"json"`, `"parquet"`, etc.).
4. **`mode("mode")`**: Define o comportamento quando os dados já existem no destino (ex.: `"overwrite"`, `"append"`).
5. **`save("path")`**: Define o caminho onde os dados serão salvos.

#### Tipos de mode

- `overwrite`: Sobrescreve o arquivo ou tabela existente.
- `append`: Adiciona novos dados ao arquivo ou tabela existente.
- `ignore`: Não realiza nenhuma ação se o arquivo ou tabela já existir.
- `error` (padrão): Lança um erro se o arquivo ou tabela já existir.

#### Salvando como CSV

**Preparando os dados**


In [0]:
spark

In [0]:

# Massa de dados
data = [
    (1, "Alice", 25, 50000),
    (2, "Bob", 30, 60000),
    (3, "Charlie", 35, 70000),
    (4, "David", 40, 80000),
    (5, "Eve", 45, 90000)
]
columns = ["id", "name", "age", "salary"]

# Criar o DataFrame
massa_dados_df = spark.createDataFrame(data, columns)

# Mostrar os dados
print("Exibindo o DataFrame:")
display(massa_dados_df)


**Salvar DataFrame como CSV**

> O caminho default do spark-warehouse é: /user/hive/warehouse


In [0]:
(
massa_dados_df
    .repartition(1)
    .write
    .format("csv")
    .mode("overwrite")
    .option("header", "true")
    .option("delimiter", ",")
    .save(f"/Volumes/workspace/default/laboratorio-spark//exemplo_csv")
)

In [0]:
display(dbutils.fs.ls(f"/Volumes/workspace/default/laboratorio-spark/exemplo_csv"))

Documentação de referência: https://spark.apache.org/docs/3.5.2/sql-data-sources-csv.html

- **Opções**:
  - `"header"`: Especifica se a primeira linha será o cabeçalho.
  - `"delimiter"`: Define o delimitador de campo (padrão é ,).

#### Salvando como JSON


In [0]:
(
    massa_dados_df
    .write
    .format("json")
    .mode("overwrite")
    .option("compression", "gzip")
    .save(f"/Volumes/workspace/default/laboratorio-spark/exemplo_json")
)

In [0]:
display(dbutils.fs.ls(f"/Volumes/workspace/default/laboratorio-spark//exemplo_json"))


Documentação de referência: https://spark.apache.org/docs/3.5.2/sql-data-sources-json.html

- **Opções**:
  - `"compression"`: Define o tipo de compressão (ex.: `"gzip"`, `"bzip2"`, `"snappy"`).

#### Salvando como Parquet

O formato **Parquet** é otimizado para análise de grandes volumes de dados. Ele suporta particionamento de dados para melhorar a eficiência das consultas.

**Preparando a massa de dados**

In [0]:
import random


# Funções auxiliares para gerar dados aleatórios
def random_name():
    names = ["Alice", "Bob", "Charlie", "David", "Eve", "Frank", "Grace", "Heidi", "Ivan", "Judy"]
    return random.choice(names)

def random_age():
    return random.randint(20, 60)

def random_salary():
    return random.randint(30000, 100000)

def random_year():
    return random.choice([2021, 2022, 2023, 2024])

def random_month():
    return random.randint(1, 12)

def random_day():
    return random.randint(1, 28)

# Gerar massa de dados
data = [(i, random_name(), random_age(), random_salary(), random_year(), random_month(), random_day()) for i in range(1, 2001)]

# Definir as colunas
columns = ["id", "name", "age", "salary", "year", "month", "day"]

# Criar o DataFrame
parquet_df = spark.createDataFrame(data, columns)

# Mostrar os dados
print("Exibindo os primeiros registros do DataFrame:")
display(parquet_df)

In [0]:
(
    parquet_df
    .write
    .format("parquet")
    .mode("overwrite")
    .partitionBy("year", "month")
    .save(f"/Volumes/workspace/default/laboratorio-spark/exemplo_parquet")
)

In [0]:
display(dbutils.fs.ls(f"/Volumes/workspace/default/laboratorio-spark/exemplo_parquet/year=2024/month=9/"))


- **Opções**:
    - `partitionBy`: Divide os dados em subdiretórios com base nos valores de uma ou mais colunas. Isso melhora a leitura seletiva dos dados.
        - Exemplo: `partitionBy("year", "month")` criará subdiretórios como `year=2024/month=09/`.
    - `bucketBy`: Agrupa os dados em "buckets" com base em uma ou mais colunas. Deve ser usado com **`saveAsTable`**.
        - Exemplo: `bucketBy(10, "id")` agrupa os dados em 10 buckets com base na coluna `id`.




#### Salvando como tabela interna

Uma **tabela interna** é gerenciada pelo Spark e armazenada no espaço de armazenamento do Spark. Essas tabelas estão geralmente no diretório padrão do warehouse do Spark.

In [0]:
(
    parquet_df
    .write
    .format("delta") # no ambiente serverless nao suporta parquet puro
    .mode("overwrite")
    .partitionBy("year", "month")
    # .bucketBy(100, "id") # quando o formato é parquet
    .saveAsTable("default.internal_table")
)

In [0]:
%sql
drop table default.internal_table


- **Opções**:
    - `partitionBy`: Divide os dados em subdiretórios com base nos valores de uma ou mais colunas. Isso melhora a leitura seletiva dos dados.
    - Exemplo: `partitionBy("year", "month")` criará subdiretórios como `year=2024/month=09/`.
    - `bucketBy`: Agrupa os dados em "buckets" com base em uma ou mais colunas. Deve ser usado com **`saveAsTable`**.
    - Exemplo: `bucketBy(10, "id")` agrupa os dados em 10 buckets com base na coluna `id`.

#### Salvando como Tabela Externa

Uma **tabela externa** permite que os dados sejam armazenados em um caminho especificado pelo usuário. O Spark gerencia apenas os metadados.


In [0]:
(
    parquet_df
    .write
    .format("parquet")
    .mode("overwrite")
    .option("path", f"/Volumes/workspace/default/laboratorio-spark/external_table__")
    .partitionBy("year", "month")
    .saveAsTable("default.external_table")
)


- **Nota**: Aqui o caminho para a tabela é especificado com a opção `"path"`, o que permite que os dados sejam armazenados externamente ao warehouse do Spark.




In [0]:
display(dbutils.fs.ls(f"/Volumes/workspace/default/laboratorio-spark/external_table__"))

In [0]:
%sql
drop table external_table

In [0]:
display(dbutils.fs.ls(f"/Volumes/workspace/default/laboratorio-spark/external_table__"))

`[INFO]: FIM DO NOTEBOOK`